In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
X = pd.read_csv("./X_train.csv", index_col=0)
y = pd.read_csv("y_train.csv", index_col = 0)

In [4]:
print(X.columns)
print(y)

Index(['Season', 'Seed_A', 'Seed_B'], dtype='object')
      y
0     1
1     0
2     1
3     1
4     0
...  ..
2446  1
2447  0
2448  0
2449  0
2450  1

[2451 rows x 1 columns]


In [5]:
le = LabelEncoder()
cat_vars = ["Seed_A", "Seed_B"]
for vars in cat_vars:
    X[vars] = le.fit_transform(X[vars])

In [6]:
kf = KFold(n_splits = 3)

In [7]:
param_dict = {"LinearModel": [0.0001, 0.0005, 0.001, 0.005, 0.01]}

In [8]:
algo_dict =  {"LinearModel": LogisticRegressionCV(cv=5, random_state=0)}

In [10]:
result_list = []
for fold_id, idx in enumerate(kf.split(X)):
    index_dict = dict(zip(['train', 'test'], idx))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": X.loc[index_vec, :],
            "y": y.loc[index_vec, :]
        }
    for algo_names, algos in algo_dict.items():
        loss_df = algos.fit(set_data_dict["train"]["X"], set_data_dict["train"]["y"])
    pred_dict = {}
    for names, models in algo_dict.items():
        pred_dict[names] = models.predict(set_data_dict["test"]["X"])
        values, counts = np.unique(set_data_dict['train']['y'], return_counts=True)
        pred_dict['featureless'] = np.repeat(values[np.argmax(counts)],
                                             len(set_data_dict["test"]["y"]))
        for algorithm, pred_vec in pred_dict.items():
            iter_err_list = {
                "test_accuracy_percent": accuracy_score(set_data_dict['test']['y'], pred_vec),
                "fold_id": fold_id,
                "algorithm": algorithm
            }
            result_list.append(pd.DataFrame(iter_err_list, index=[0]))
iter_err_df = pd.concat(result_list)
test_acc_df =  iter_err_df.groupby(['algorithm'])['test_accuracy_percent'].mean().reset_index()
# print(test_acc_df)

C:\Users\kyler\anaconda3\envs\python_env\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kyler\anaconda3\envs\python_env\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


     algorithm  test_accuracy_percent
0  LinearModel               0.692370
1  featureless               0.493268


C:\Users\kyler\anaconda3\envs\python_env\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
iter_err_df.to_csv("./model_results.csv")